In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from IPython.display import Image

In [3]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [4]:
from we import get_we, initiate_model

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from scipy.stats import f_oneway

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [10]:
labels = [m['label'] for m in models]

In [11]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_nouns_we.csv'
                    )

In [12]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,Number,Gender,Lemma,Semantic
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.836139,0.551025,0.347197,0.789782,0.393246,0.300031,0.609232,0.913722,0.178287,0.838988,...,0.586894,0.674134,0.276217,0.081619,0.154076,0.553278,invariable,feminine,2D,Other
3D,0.828946,0.549880,0.339958,0.797102,0.395346,0.301288,0.603051,0.916539,0.185836,0.844714,...,0.584636,0.674752,0.278596,0.078074,0.143366,0.548315,invariable,feminine,3D,Other
a,0.826660,0.556389,0.332618,0.805891,0.399844,0.295803,0.597985,0.914675,0.201301,0.846672,...,0.577705,0.673679,0.278649,0.081689,0.139490,0.533415,invariable,masculine,a,Attribute
aa,0.826916,0.555891,0.332298,0.805754,0.400046,0.296797,0.597758,0.914183,0.201098,0.845985,...,0.578594,0.673429,0.278861,0.081218,0.139010,0.533594,invariable,masculine,aa,Other
abandon,0.826281,0.556306,0.332007,0.805256,0.399368,0.296458,0.597567,0.914983,0.200809,0.846577,...,0.578808,0.673362,0.279152,0.081560,0.139390,0.534123,singular,masculine,abandon,Other


In [13]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'Semantic'

In [32]:
we_with_features[9][we_with_features[9].Semantic == 'Person']

,0,1,2,3,4,5,6,7,8,9,...,762,763,764,765,766,767,Number,Gender,Lemma,Semantic
Word,,,,,,,,,,,,,,,,,,,,,
adjoint,0.823055,0.554145,0.331061,0.798859,0.403883,0.305467,0.598045,0.918237,0.200484,0.846411,...,0.580579,0.666723,0.285854,0.078263,0.137106,0.542804,singular,masculine,adjoint,Person
adolescente,0.825379,0.556263,0.332963,0.801826,0.399729,0.295553,0.597687,0.915818,0.200318,0.845772,...,0.581678,0.670208,0.280697,0.080918,0.137876,0.536667,singular,feminine,adolescente,Person
adultes,0.826321,0.553359,0.340098,0.801048,0.402006,0.299143,0.600195,0.919297,0.197051,0.843050,...,0.582661,0.672207,0.282385,0.082297,0.139731,0.541618,plural,invariable,adulte,Person
amateurs,0.825079,0.554154,0.332445,0.802781,0.401279,0.299754,0.598612,0.920054,0.198992,0.847277,...,0.580442,0.671620,0.282495,0.081704,0.138196,0.537826,plural,masculine,amateur,Person
ambassadeur,0.825435,0.556263,0.331126,0.803983,0.400720,0.298471,0.597692,0.915810,0.202349,0.845497,...,0.580679,0.672751,0.280438,0.080750,0.138613,0.535024,singular,masculine,ambassadeur,Person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
élu,0.826157,0.556126,0.331690,0.805167,0.399991,0.297568,0.597749,0.914584,0.201295,0.845682,...,0.578993,0.673380,0.279232,0.080850,0.138975,0.534183,singular,masculine,élu,Person
épouse,0.826211,0.556320,0.332777,0.805629,0.400047,0.296790,0.597891,0.914350,0.201516,0.846018,...,0.577675,0.673093,0.278377,0.081435,0.138604,0.533941,singular,feminine,épouse,Person
époux,0.826079,0.555657,0.332092,0.803980,0.399801,0.297693,0.599023,0.916135,0.201509,0.845720,...,0.579990,0.673340,0.279966,0.081429,0.138269,0.535503,invariable,masculine,époux,Person


# Nouns

In [20]:
pos = ['NOUN']

Split each model into train and test using k_fold cross validation:

In [21]:
X_noun_train = []
y_noun_train = []

X_noun_test = []
y_noun_test = []

In [22]:
n_folds = 5

In [23]:
for we in we_with_features:
    X, y = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='Act',
                                           split=False,
                                           balance=True)
    X_trains = []
    y_trains = []
    
    X_tests = []
    y_tests = []
    
    X_folds = np.array_split(X, n_folds)
    y_folds = np.array_split(y, n_folds)
    
    for i in range(n_folds):
        X_trains.append(pd.DataFrame(np.concatenate(X_folds[:i] + X_folds[i+1:])))
        y_trains.append(np.concatenate(y_folds[:i] + y_folds[i+1:]))

        X_folds[i].columns = X_folds[i].columns.map(int)
        X_tests.append(X_folds[i])
        y_tests.append(y_folds[i])
        
    
    X_noun_train.append(X_trains)
    X_noun_test.append(X_tests)
    
    y_noun_train.append(y_trains)
    y_noun_test.append(y_tests)

In [24]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['1 fold size'] = [len(x[0]) for x in X_noun_test]
dataset_sizes

,1 fold size
flau_small_c,159
flau_base_u,163
flau_base_c,159
flau_large_c,159
cam_base,122
xlm_large,42
xlm_base,42
bert_base_u,71
distilbert_base,62
bert_base_c,62


## Non-independent dims

Calculate non-independendent dimensions for each of _k_ splits.

ANOVA test with p-value < 0.01:

In [25]:
anova_dims = []

In [26]:
for i in range(len(models)):
    anova_dims.append([])
    for j in range(n_folds):
        anova_dims[i].append(get_anova_dims(X_noun_train[i][j], y_noun_train[i][j]))

Get dimensions where MI > 0:

In [27]:
mi_dims = []

In [28]:
for i in range(len(models)):
    mi_dims.append([])
    for j in range(n_folds):
        mi_dims[i].append(get_mi_dims(X_noun_train[i][j], y_noun_train[i][j]))

In [29]:
for i in range(len(models)):
    for j in range(n_folds):
        temp = list(map(lambda x: int(x), mi_dims[i][j]))
        mi_dims[i][j] = temp

Get dimensions that are both found by the ANOVA independency test and MI test:

In [30]:
non_ind_dims = []

In [31]:
for i in range(len(models)):
    non_ind_dims.append([])
    for j in range(n_folds):
        non_ind_dims[i].append(list(set(anova_dims[i][j]).intersection(mi_dims[i][j])))

Stats about the number of dimensions for each model for the 1st split:

In [32]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [33]:
non_ind_df['ANOVA'] = [len(x[0]) for x in anova_dims]
non_ind_df['MI'] = [len(x[0]) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x[0]) for x in non_ind_dims]

In [34]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,198,303,149
flau_base_u,77,419,57
flau_base_c,97,407,60
flau_large_c,324,606,239
cam_base,23,397,15
xlm_large,70,661,54
xlm_base,6,456,5
bert_base_u,2,395,1
distilbert_base,60,442,47
bert_base_c,33,416,26


In [35]:
dims = {}

In [36]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = [X_noun_train[i][0].columns]*n_folds
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [37]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [38]:
lr_res = []

In [39]:
for i in range(len(models)):
    lr_res.append([])
    for j in range(n_folds):
        lr_res[i].append(lr(X_noun_train[i][j], y_noun_train[i][j]))

Train Perceptron:

In [40]:
perceptron_res = []

In [41]:
for i in range(len(models)):
    perceptron_res.append([])
    for j in range(n_folds):
        perceptron_res[i].append(perceptron(X_noun_train[i][j], y_noun_train[i][j]))

Compute correlation to the gender vector:

In [42]:
corr_res = []

In [43]:
for i in range(len(models)):
    corr_res.append([])
    for j in range(n_folds):
        corr_res[i].append(correlation(X_noun_train[i][j], y_noun_train[i][j]))

In [44]:
for i in range(len(models)):
    for alpha in alphas:
        dims[labels[i]][f'LR{alpha}'] = []
        dims[labels[i]][f'Perc{alpha}'] = []
        dims[labels[i]][f'Corr{alpha}'] = []
        dims[labels[i]][f'All imp dims{alpha}'] = []
        for j in range(n_folds):
            num_imp_dims = len(X_noun_test[i][0].columns)*alpha//100
            lr_dims = [x[0] for x in lr_res[i][j][:num_imp_dims]]
            perc_dims = [x[0] for x in perceptron_res[i][j][:num_imp_dims]]
            corr_dims = [x[0] for x in corr_res[i][j][:num_imp_dims]]
            dims[labels[i]][f'LR{alpha}'].append(lr_dims)
            dims[labels[i]][f'Perc{alpha}'].append(perc_dims)
            dims[labels[i]][f'Corr{alpha}'].append(corr_dims)
            dims[labels[i]][f'All imp dims{alpha}'].append(list(set(lr_dims).intersection(perc_dims).intersection(corr_dims)))

In [45]:
dims[labels[0]]['LR1']

[[294, 499, 423, 201, 158],
 [313, 294, 95, 423, 485],
 [294, 245, 119, 281, 95],
 [294, 485, 113, 33, 202],
 [294, 485, 255, 281, 113]]

## Compute medians

In [46]:
medians = {}

In [47]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else list(dims[labels[i]][dim_group])
            # Median of dimensions where feature vector is equal to 0
            median_0 = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_list].median()
            # Median of dimensions where feature vector is equal to 1
            median_1 = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_list].median()

            medians[labels[i]][dim_group].append({
                    '0': median_0,
                    '1': median_1
                })

We can compare the number of dimensions found by each test for each model in the first fold:

In [48]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group][0]) if dim_group != 'All dims' else \
                len(dims[model][dim_group])

In [49]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,5,5,5,5,5,5,5,5,5,5
ANOVA,198,77,97,324,23,70,6,2,60,33
MI,303,419,407,606,397,661,456,395,442,416
All non ind,149,57,60,239,15,54,5,1,47,26
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,1,0,0,1,1,0,1,0,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [50]:
y_preds = {}

In [51]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_preds[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else dims[labels[i]][dim_group]
            
            # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
            mae0 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['0'], x), axis=1)
            mae1 = X_noun_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['1'], x), axis=1)
            
            y_preds[labels[i]][dim_group].append((mae0 > mae1).apply(int))
    

## Compute accuracies

In [52]:
accs = {}

In [53]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        accs[labels[i]][dim_group] = []
        for j in range(n_folds):
            y_true = y_noun_test[i][j]
            y_pred = y_preds[labels[i]][dim_group][j]
            if any(y_pred):
                acc = accuracy_score(y_true, y_pred)
            else:
                acc = 0
            accs[labels[i]][dim_group].append(acc)

In [54]:
# Compute average accuracy between 5 folds
avg_accs = {}

In [55]:
for i in range(len(models)):
    avg_accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        avg_accs[labels[i]][dim_group] = np.average(accs[labels[i]][dim_group])

In [56]:
accs_df = pd.DataFrame(avg_accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.780225,0.593358,0.635117,0.786585,0.526446,0.528571,0.542857,0.457827,0.614384,0.548969
ANOVA,0.782740,0.599470,0.623772,0.773927,0.534670,0.533333,0.442857,0.287324,0.604548,0.470122
MI,0.762559,0.582292,0.627522,0.777749,0.523140,0.528571,0.547619,0.460644,0.630619,0.548969
All non ind,0.768864,0.587245,0.636343,0.772661,0.521420,0.561905,0.342857,0.261972,0.614384,0.463617
LR1,0.662845,0.507347,0.551747,0.631303,0.508224,0.523810,0.542857,0.449376,0.624220,0.545743
Perc1,0.704498,0.581110,0.553037,0.694491,0.516529,0.557143,0.533333,0.457827,0.568429,0.555473
Corr1,0.698296,0.584708,0.577080,0.704530,0.542880,0.538095,0.566667,0.477787,0.624379,0.525912
All imp dims1,0.493352,0.090123,0.000000,0.374325,0.405853,0.000000,0.119048,0.000000,0.000000,0.000000
LR5,0.758793,0.556480,0.574572,0.736096,0.495055,0.509524,0.580952,0.466278,0.643839,0.549075
Perc5,0.738643,0.588419,0.560600,0.762583,0.491722,0.519048,0.547619,0.466278,0.565256,0.542411


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [57]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
ANOVA,0.002516,0.006112,-0.011345,-0.012658,0.008224,4.761905e-03,-0.100000,-0.170503,-0.009836,-0.078847
MI,-0.017666,-0.011066,-0.007595,-0.008837,-0.003306,-1.110223e-16,0.004762,0.002817,0.016235,0.000000
All non ind,-0.011361,-0.006112,0.001226,-0.013924,-0.005026,3.333333e-02,-0.200000,-0.195855,0.000000,-0.085352
LR1,-0.117379,-0.086011,-0.083369,-0.155282,-0.018222,-4.761905e-03,0.000000,-0.008451,0.009836,-0.003226
Perc1,-0.075726,-0.012247,-0.082079,-0.092095,-0.009917,2.857143e-02,-0.009524,0.000000,-0.045955,0.006504
Corr1,-0.081928,-0.008650,-0.058037,-0.082056,0.016434,9.523810e-03,0.023810,0.019960,0.009995,-0.023057
All imp dims1,-0.286872,-0.503234,-0.635117,-0.412260,-0.120593,-5.285714e-01,-0.423810,-0.457827,-0.614384,-0.548969
LR5,-0.021431,-0.036878,-0.060545,-0.050490,-0.031391,-1.904762e-02,0.038095,0.008451,0.029455,0.000106
Perc5,-0.041581,-0.004938,-0.074516,-0.024003,-0.034724,-9.523810e-03,0.004762,0.008451,-0.049127,-0.006557


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [58]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c LR25 0.02022131995860199
flau_base_u Perc25 0.013489358479133573
flau_base_c All non ind 0.0012260170368602052
flau_large_c LR75 0.007539208661730856
cam_base Corr1 0.016434087522016028
xlm_large All non ind 0.033333333333333215
xlm_base All imp dims10 0.042857142857142816
bert_base_u Corr1 0.019959758551307882
distilbert_base LR10 0.06858804865150725
bert_base_c Perc1 0.00650449497620309


In [59]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c LR25 0.80044582437704
flau_base_u Perc25 0.6068469287283194
flau_base_c All non ind 0.6363426478783537
flau_large_c LR75 0.7941246716025795
cam_base Corr1 0.5428803685137515
xlm_large All non ind 0.5619047619047619
xlm_base All imp dims10 0.5857142857142857
bert_base_u Corr1 0.4777867203219316
distilbert_base LR10 0.6829719725013221
bert_base_c Perc1 0.5554732945531465


For each fold find dimensions with the highest accuracy, and pick dimensions appeared in all 5 folds:

In [60]:
best_dims = {}

In [61]:
dim_cand_accs = {}

In [62]:
for i in range(len(labels)):
    bd = []
    for j in range(n_folds):
        best_dim_group = ''
        best_acc = 0
        for dim_group in accs[labels[i]].keys():
            if accs[labels[i]][dim_group][j] > best_acc:
                best_dim_group = dim_group
                best_acc = accs[labels[i]][dim_group][j]
        bd.extend(dims[labels[i]][best_dim_group][j])
    
    # Find dimensions that appeared within dimensions with the highest accuracy in all 5 folds
    d, c = np.unique(bd, return_counts=True)
    dim_cand = [x[0] for x in zip(d, c) if x[1] >= 5]
    best_dims[labels[i]] = dim_cand
    print(f'{labels[i]}: {len(dim_cand)} dimensions repeated in 5 folds')

    # For each fold build a prediction and calculate accuracy
    cand_accs = []
    for j in range(n_folds):
        med0_cand = X_noun_train[i][j][y_noun_train[i][j] == 0][dim_cand].median()
        med1_cand = X_noun_train[i][j][y_noun_train[i][j] == 1][dim_cand].median()
        
        mae0_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med0_cand, x), axis=1)
        mae1_cand = X_noun_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med1_cand, x), axis=1)
        
        y_pred_cand = (mae0_cand > mae1_cand).apply(int)
        if any(y_pred_cand):
            cand_accs.append(accuracy_score(y_true=y_noun_test[i][j], y_pred=y_pred_cand))
        else:
            cand_accs.append(0)
    dim_cand_accs[labels[i]] = cand_accs
    print(f'Average accuracy: {np.average(cand_accs)}')

flau_small_c: 48 dimensions repeated in 5 folds
Average accuracy: 0.8055170766658705
flau_base_u: 2 dimensions repeated in 5 folds
Average accuracy: 0.5945769900780127
flau_base_c: 14 dimensions repeated in 5 folds
Average accuracy: 0.6388822545975639
flau_large_c: 50 dimensions repeated in 5 folds
Average accuracy: 0.7991720404426399
cam_base: 1 dimensions repeated in 5 folds
Average accuracy: 0.5544777130470127
xlm_large: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_base: 1 dimensions repeated in 5 folds
Average accuracy: 0.5904761904761905
bert_base_u: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
distilbert_base: 3 dimensions repeated in 5 folds
Average accuracy: 0.6047065044949761
bert_base_c: 0 dimensions repeated in 5 folds
Average accuracy: 0.0


In [63]:
with open('../Data/best_results/semantic_act_dims.pickle', 'wb') as f:
    pickle.dump(dims, f)

In [64]:
with open('../Data/best_results/semantic_act_accs.pickle', 'wb') as f:
    pickle.dump(accs, f)

In [65]:
with open('../Data/best_results/semantic_act_medians.pickle', 'wb') as f:
    pickle.dump(medians, f)

# Conclusion